In [2]:
# Big Bang Theory 웹 크롤링

## Library import 및 Soup 가져오기 함수 정의
- Big Bang Theory 시즌별 에피소드가 있는 페이지를 가져온 후 시즌을 추출

In [3]:
import urllib
from urllib.request import urlopen
from urllib.request import Request
from bs4 import BeautifulSoup
import pandas as pd
import re

def getHtmlFromUrl(url):
    req = Request(url=url, headers={"User-agent":"Mozilla/5.0"})
    res = urlopen(req)
    return BeautifulSoup(res, "html.parser")
prefix = "https://www.imdb.com/title/tt0898266/episodes/?"

html = getHtmlFromUrl(prefix)

season_html = html.find(class_="ipc-tabs ipc-tabs--base ipc-tabs--align-left").find_all('a')

def getSeasonFromHtml(html):
    season = []
    for s in html:
        season.append(s.string)
    return season
season = getSeasonFromHtml(season_html)

pattern = "^S[0-9]+.E[0-9]+"
season


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', 'Unknown']

## 필요 함수 정의

In [6]:
episodeNameClass = "sc-56c21e9b-0 eoThov"

def getEpisodeHtml(html):
    return html.find(class_=episodeNameClass).find_all("article")

def getEpisodeNameFromHtml(episodeHtml):
    return episodeHtml.find(class_="ipc-title__text").getText()

def getRateFromHtml(episodeHtml):
    return episodeHtml.find(class_="ipc-rating-star--rating").getText()

def getDateFromHtml(episodeHtml):
    return episodeHtml.find(class_="sc-f2169d65-10 bYaARM").getText()




## 데이터 추출
- 에피소드 목록 페이지에서 시즌별 에피소드, 방영날짜, 평점, season 추출 후 DataFrame으로 만듦

In [7]:
episodeList = []
dateList = []
rateList = []
seasonList = []

for seasonIdx in season:
    url = prefix + "season=" + str(seasonIdx)
    html = getHtmlFromUrl(url)
    
    for idx, episodeHtml in enumerate(getEpisodeHtml(html)):
        episodeName =  getEpisodeNameFromHtml(episodeHtml)
        episode = idx + 1
        rate = getRateFromHtml(episodeHtml)
        date = getDateFromHtml(episodeHtml)
        episodeList.append(episode)
        rateList.append(rate)
        dateList.append(date)
        seasonList.append(seasonIdx)
data = {"Season":seasonList, "Episode":episodeList, "Rate":rateList, "Date":dateList}
df = pd.DataFrame(data)
df

,Season,Episode,Rate,Date
0,1,1,8.1,"Sun, Sep 23, 2007"
1,1,2,8.2,"Sun, Sep 30, 2007"
2,1,3,7.6,"Sun, Oct 7, 2007"
3,1,4,8.0,"Sun, Oct 14, 2007"
4,1,5,7.8,"Sun, Oct 21, 2007"
...,...,...,...,...
275,12,21,7.2,"Wed, May 1, 2019"
276,12,22,7.9,"Wed, May 8, 2019"
277,12,23,9.0,"Wed, May 15, 2019"
278,12,24,9.5,"Wed, May 15, 2019"


## 에피소드 상세내용 페이지
- 상세내용 페이지로 들어간 후 Top Cast Actor 이름 추출 테스트

In [57]:
prefix = "https://www.imdb.com"
url = prefix + "/title/tt0898266/episodes/?season=" + '1'
html = getHtmlFromUrl(url)

episode = getEpisodeHtml(html)[0]
url = prefix + episode.find('a')["href"]
html = getHtmlFromUrl(url)
title = html.find(class_="hero__primary-text").string
title
topcastList = []
actors = html.find_all(class_="sc-cd7dc4b7-7 vCane")
actor = html.find(class_="sc-cd7dc4b7-7 vCane").find('a').string
topcast = ""
for idx, actor in enumerate(actors):
    actor = actor.find('a').string
    if idx == 0:
        topcast += actor.string
    else:
        topcast += (", " + actor.string)
topcastList.append(topcast)
topcastList


['Johnny Galecki, Jim Parsons, Kaley Cuoco, Simon Helberg, Kunal Nayyar, Vernee Watson, Brian Patrick Wade']

## 타이틀 및 출연자 추출
- 테스트를 바탕으로 Title 및 Top Cast 추출

In [59]:
from tqdm import tqdm
titleList = []
topcastList = []
prefix = "https://www.imdb.com"
for seasonIdx in tqdm(season, total=len(season)):
    url = prefix + "/title/tt0898266/episodes/?season=" + str(seasonIdx)
    html = getHtmlFromUrl(url)
    episodes = getEpisodeHtml(html)

    for episode in episodes:
        url = prefix + episode.find('a')["href"]
        html = getHtmlFromUrl(url)
        title = html.find(class_="hero__primary-text").string
        titleList.append(title)
        actors = html.find_all(class_="sc-cd7dc4b7-7 vCane")
        topcast = ""
        for idx, actor in enumerate(actors):
            actor = actor.find('a').string
            if idx == 0:
                topcast += actor.string
            else:
                topcast += (", " + actor.string)
        topcastList.append(topcast)

len(titleList), len(topcastList)

100%|██████████| 13/13 [10:15<00:00, 47.35s/it]


(280, 280)

## DataFrame 정리
- 추가된 내용을 DataFrame에 넣고 Column 정리

In [60]:
df["Title"] = titleList
df["Top Cast"] = topcastList
df = df.loc[:, ["Season", "Episode", "Title", "Top Cast", "Rate", "Date"]]
df.head()

,Season,Episode,Title,Top Cast,Rate,Date
0,1,1,Pilot,"Johnny Galecki, Jim Parsons, Kaley Cuoco, Simo...",8.1,8.1
1,1,2,The Big Bran Hypothesis,"Johnny Galecki, Jim Parsons, Kaley Cuoco, Simo...",8.2,8.2
2,1,3,The Fuzzy Boots Corollary,"Johnny Galecki, Jim Parsons, Kaley Cuoco, Simo...",7.6,7.6
3,1,4,The Luminous Fish Effect,"Johnny Galecki, Jim Parsons, Kaley Cuoco, Simo...",8.0,8.0
4,1,5,The Hamburger Postulate,"Johnny Galecki, Jim Parsons, Kaley Cuoco, Simo...",7.8,7.8
